In [1]:
from selenium import webdriver
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
url = "https://google.com"
driver = webdriver.Chrome(executable_path="C:\\Users\\дом.DESKTOP-A3U9TO1\\DataspellProjects\\moretech_prokhodilimimo\\chromedriver\\chromedriver.exe")
try:
    driver.get(url=url)
    time.sleep(10)
except Exception as e:
    print(e)
finally:
    driver.close()
    driver.quit()


C:\Users\дом.DESKTOP-A3U9TO1\AppData\Local\Temp\ipykernel_13036\2303112630.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:\\Users\\дом.DESKTOP-A3U9TO1\\DataspellProjects\\moretech_prokhodilimimo\\chromedriver\\chromedriver.exe")


WebDriverException: Message: disconnected: unable to connect to renderer
  (failed to check if window was closed: disconnected: unable to connect to renderer)
  (Session info: chrome=106.0.5249.103)
Stacktrace:
Backtrace:
	Ordinal0 [0x00DF1ED3+2236115]
	Ordinal0 [0x00D892F1+1807089]
	Ordinal0 [0x00C966FD+812797]
	Ordinal0 [0x00C8798A+752010]
	Ordinal0 [0x00C7C754+706388]
	Ordinal0 [0x00C7D564+709988]
	Ordinal0 [0x00CEA655+1156693]
	Ordinal0 [0x00CE1A53+1120851]
	Ordinal0 [0x00CBA73D+960317]
	Ordinal0 [0x00CBB71F+964383]
	GetHandleVerifier [0x0109E7E2+2743074]
	GetHandleVerifier [0x010908D4+2685972]
	GetHandleVerifier [0x00E82BAA+532202]
	GetHandleVerifier [0x00E81990+527568]
	Ordinal0 [0x00D9080C+1837068]
	Ordinal0 [0x00D94CD8+1854680]
	Ordinal0 [0x00D94DC5+1854917]
	Ordinal0 [0x00D9ED64+1895780]
	BaseThreadInitThunk [0x75E16739+25]
	RtlGetFullPathName_UEx [0x77158FD2+1218]
	RtlGetFullPathName_UEx [0x77158F9D+1165]


In [6]:
pip install lxml

     ---------------------------------------- 3.6/3.6 MB 2.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [66]:
import requests
from bs4 import BeautifulSoup
import json


#URL = 'https://www.rbc.ru/neweconomy/'
URL = 'https://www.rbc.ru/'


def get_soup(url):
    r = requests.get(url).text
    return BeautifulSoup(r, 'lxml')


def save_json(data):
    with open('rbk_data1.json', "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)


def main():
    news_data = {}

    soup = get_soup(URL)

    # Получаем все ссылки на новости
    news_links = soup.find('div', class_='main').find_all('a', {'class': ['main__big__link', 'main__feed__link']})

    # Для каждой ссылки получаем информацию и записываем в news_data
    for i in range(12):

        link = news_links[i].get('href').split('?')[0]
        name = link
        news_data[name] = {}
        soup = get_soup(link)

        # Переходим на страницу для дальнейшенго парсинга
        article = soup.find('div', class_='article')
        #print(article.find('a'))
        category = article.find('a', class_='article__header__category')

        #date = article.find('span', class_='article__header__date').get('content').replace('T', ' ').split('+')[0]
        title = article.find('span', class_='js-slide-title')
        image = article.find('div', class_='article__main-image')

        article_paragraphs = article.find_all('p')
        article_text = ''
        for paragraph in article_paragraphs:
            article_text += paragraph.text

        # Заполняем полученными данными news_data
        news_data[name]['link'] = link
        #news_data[name]['date'] = date
        news_data[name]['text'] = article_text.replace('\xa0', '').replace('\n', '').replace('\r', '')

        try:
            news_data[name]['title'] = title.text
        except AttributeError:
            news_data[name]['title'] = 'Без заголовка'
        try:
            news_data[name]['category'] = category.text.replace('\n', '')
        except AttributeError:
            news_data[name]['category'] = 'Без категории'
        try:
            news_data[name]['image'] = image.find('img').get('src')
        except AttributeError:
            news_data[name]['image'] = 'Без обложки'

    save_json(news_data)


if __name__ == "__main__":
    main()

In [63]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import json


URL = 'https://www.rbc.ru/neweconomy/'
#URL = 'https://www.rbc.ru/'


def get_soup(url):
    r = requests.get(url).text
    return BeautifulSoup(r, 'lxml')


def save_json(data):
    with open('rbk_data.json', "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

def main():
    news_data = {}

    soup = get_soup(URL)

    # Получаем все ссылки на новости
    #print(soup.find_all('a', {'class' : ['q-item__link', ]}))

    news_links = soup.find_all('a', {'class' : ['q-item__link', ]})

    # Для каждой ссылки получаем информацию и записываем в news_data
    for i in range(len(news_links)):

        link = news_links[i].get('href').split('?')[0]
        name = link
        news_data[name] = {}
        soup = get_soup(link)
        #print(soup.text)

        # Переходим на страницу для дальнейшенго парсинга
        article = soup.find('div', class_='article')
        #print(article)

        #print(article.find('a'))
        #category = article.find('a', class_='article__header__category')
        #date = article.find('span', class_='atricle__date-update').get('content').replace('T', ' ').split('+')[0]
        try:
            date = soup.find('div', class_='atricle__date-update').get_text().strip().split()[1]
        except AttributeError:
            continue
        print(date)
        title = soup.title
        #image = article.find('div', class_='article__main-image')
        try:
            article_paragraphs = article.find_all('p')
        except AttributeError:
            continue
        article_text = ''
        for paragraph in article_paragraphs:
            article_text += paragraph.text
        #print(article_text)

        # Заполняем полученными данными news_data
        news_data[name]['link'] = link
        news_data[name]['date'] = date
        news_data[name]['text'] = article_text.replace('\xa0', '').replace('\n', '').replace('\r', '')

        try:
            news_data[name]['title'] = title.text
        except AttributeError:
            news_data[name]['title'] = 'Без заголовка'
        #try:
            #news_data[name]['category'] = category.text.replace('\n', '')
        #except AttributeError:
            #news_data[name]['category'] = 'Без категории'
        #try:
            #news_data[name]['image'] = image.find('img').get('src')
        #except AttributeError:
            #news_data[name]['image'] = 'Без обложки'

    save_json(news_data)


if __name__ == "__main__":
    main()

07.10.2022
07.10.2022
07.10.2022
07.10.2022
06.10.2022
04.10.2022
04.10.2022
03.10.2022
28.09.2022
29.09.2022
29.09.2022
27.09.2022
28.09.2022
26.09.2022
26.09.2022
20.09.2022
16.09.2022
14.09.2022
15.09.2022
07.09.2022
06.09.2022
01.09.2022
29.08.2022
25.08.2022
